In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

In [27]:
np.set_printoptions(threshold=10000)

In [24]:
def import_outside_air_temperature_d_t(area_number) -> np.ndarray:
    """
    日平均外気温度を外部CSVファイルからND配列に読み込む
    Returns:
        outside_air_temperature_d_t : 外気温度[8760], ℃ 
    """
    df = pd.read_csv('climateData_' + str(area_number) + '.csv', encoding='SHIFT_JIS')
    ds = df['外気温[℃]']
    outside_air_temperature_d_t = ds.values
    return outside_air_temperature_d_t

In [28]:
import_outside_air_temperature_d_t(5)

array([ -0.9,  -2. ,  -2.8,  -4. ,  -4.2,  -4.6,  -4.7,  -2. ,   1.2,
         3.7,   5.1,   5.7,   6.3,   6.5,   6.2,   5. ,   3.6,   2.9,
         1.2,   1.1,  -0.7,  -1.5,  -2.4,  -3.3,  -3.5,  -4.1,  -5. ,
        -5.1,  -4.7,  -5.5,  -5.6,  -3.7,   0.3,   2.4,   3.8,   5.1,
         5.5,   5.5,   5.5,   5. ,   2.7,   1.4,   0.1,  -1.5,  -1.7,
        -1.8,  -1.8,  -1.9,  -3.2,  -4. ,  -3.4,  -5.1,  -4.8,  -4.6,
        -5.5,  -1.9,   0.7,   3.3,   4.6,   5.7,   6.1,   7. ,   6.9,
         6.5,   4.5,   3.8,   3.7,   3.6,  -0.6,  -1. ,  -1.7,  -2. ,
        -3.4,  -3.7,  -4.3,  -5.2,  -5.5,  -5.1,  -5.8,  -2.8,   1.7,
         3.5,   4.8,   5.2,   6.1,   6.5,   6.6,   5.9,   3.7,   1.1,
        -0.3,  -0.9,  -1.6,  -1.7,  -1.8,  -1.3,   0. ,   0.4,   0.3,
         0.1,  -0.4,  -0.1,  -1.3,  -0.7,   1. ,   4.3,   5.2,   6.2,
         6.4,   7.2,   8.2,   7. ,   4.1,   4.3,   4.9,   5.5,   5.9,
         5.7,   4.6,   4.5,   4.5,   4.6,   4.3,   4.9,   4.9,   4.6,
         4.7,   5. ,

# (自立)換気設備
自立循環型住宅の試算では、換気設備は省エネ基準の計算過程において時刻別比消費電力を置き換えることで評価する。

### 時刻別比消費電力
$$
f_{ SFP, d, t }
=
\begin{cases}
f_{ SFP }
& ,
\mbox{制御しない}
\\
f_{ SFP, single, d, t }
& ,
\mbox{片側運転}
\\
f_{ SFP, by-pass, d, t }
& ,
\mbox{バイパス運転}
\\
f_{ SFP, hybrid-temperature, d, t }
& ,
\mbox{温度差利用型ハイブリッド運転}
\end{cases}
$$

$ f_{ SFP, d, t } $：日付dの時刻tにおける比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP } $：比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP, single, d, t } $：日付dの時刻tにおける片側運転機能を有する換気設備の比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP, by-pass, d, t } $：日付dの時刻tにおけるバイパス運転機能を有する換気設備の比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP, hybrid-temperature, d, t } $：日付dの時刻tにおける温度差利用型ハイブリット運転機能を有する換気設備のの比消費電力,W/(m<sup>3</sup>/h)   

In [40]:
def get_SFP_d_t(ventilation_control_mode, f_SFP, f_SFP_bypass, \
                HeatingSeason_d, \
                area_number, \
                total_floor_area, \
                val_max, tolerance, maximum_iterations, \
                stories, \
                room_air_temperature, \
                c_value, \
                ventilation_type):
    
    """時刻別消費電力量を計算する
    
    Parameters
    ----------
    ventilation_cntrol_mode : strings
        換気設備の制御方式（制御しない、片側運転、バイパス運転、温度差利用型ハイブリッド運転）,--  
        
    f_SFP : float
        比消費電力,W/(m^3/h)  

    f_SFP_bypass : float
        バイパス運転時の比消費電力,W/(m^3/h)  
        
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
          
    area_number : int
        地域区分番号,--  
    
    total_floor_area : float
        延べ床面積,m^2
        
    val_min : float
        二分法における初項（最小値）,Pa  
    
    val_max : float
        二分法における初項（最大値）,Pa  
    
    tolerance : float
        二分法における許容誤差,Pa  

    maximum_iterations : float
        二分法における繰り返し計算の上限回数,回  
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
    
    room_air_temperature : float
        室温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,- 
        
    Returns
    ----------
    SFP_d_t : ndarray(N-dimensional array)
        1年間の全時間の比消費電力量を格納したND配列,W/(m^3/h)  
        d日t時の比消費電力量が年開始時から8760個連続して格納されている
    """
    
    if ventilation_control_mode == "制御しない":
        SFP_d_t = f_SFP
        
    elif ventilation_control_mode == "片側運転":
        SFP_d_t = get_SFP_single_d_t(f_SFP, HeatingSeason_d)
    
    elif ventilation_control_mode == "バイパス運転":
        SFP_d_t = get_SFP_bypass_d_t(f_SFP, f_SFP_bypass, HeatingSeason_d)
    
    elif ventilation_control_mode == "温度差利用型ハイブリッド運転":
        SFP_d_t = get_SFP_hybrid_temperature_d_t( \
                        f_SFP, \
                        HeatingSeason_d, \
                        area_number, \
                        total_floor_area, \
                        val_max, tolerance, maximum_iterations, \
                        stories, \
                        room_air_temperature, \
                        c_value, \
                        ventilation_type \
                    )
    
    else:
        raise Exception
    
    return SFP_d_t

## 片側運転機能を有する換気設備の比消費電力

### 時刻別比消費電力
$$
f_{ SFP, single, d, t }
=
f_{ SFP }
\times
C_{ single, d, t }
$$

$$
C_{ single, d, t }
=
\begin{cases}
0.58
& ,
HeatingSeason_{ d } = \mbox{False}
\\
1
& ,
HeatingSeason_{ d } = \mbox{True}
\end{cases}
$$

$ f_{ SFP, single, d, t } $：日付dの時刻tにおける片側運転機能を有する換気設備の比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP } $：換気設備の比消費電力,W/(m<sup>3</sup>/h)    
$ C_{ single, d, t } $：日付dの時刻tにおける片側運転の比消費電力の補正係数,-  
$ HeatingSeason_{ d } $：日付dが暖房期である,True/False   

In [4]:
def get_SFP_single_d_t(f_SFP, HeatingSeason_d):
    """片側運転機能有する換気設備の時刻別比消費電量を計算する
    
    Parameters
    ----------
    f_SFP : float
        比消費電力,W/(m^3/h)  
        
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
    
    Returns
    ----------
    SFP_single_d_t : ndarray(N-dimensional array)
        1年間の全時間の比消費電力量を格納したND配列, kWh
        d日t時の消費電力量が年開始時から8760個連続して格納されている
    """
    
    C_d_t = np.where(np.repeat(HeatingSeason_d, 24) == True, 1, 0.58)
    
    SFP_single_d_t = f_SFP * C_d_t
    
    return SFP_single_d_t

In [5]:
get_SFP_single_d_t(0.3, np.tile([True, False, True, False, True], 73))

array([ 0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,  0.174,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
        0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,  0.3  ,
      

## バイパス運転を有する換気設備の比消費電力

### 時刻別比消費電力
$$
f_{ SFP, by-pass, d, t }
=
\begin{cases}
f_{ SFP }
& ,
HeatingSeason_{ d } = \mbox{True}
\\
f_{ SFP, by-pass }
& ,
HeatingSeason_{ d } = \mbox{False}
\end{cases}
$$

$ f_{ SFP, by-pass, d, t } $：日付dの時刻tにおけるバイパス運転機能を有する換気設備の比消費電力,W/(m<sup>3</sup>/h)  
$ f_{ SFP } $：換気設備の比消費電力,W/(m<sup>3</sup>/h)    
$ f_{ SFP, by-pass } $：換気設備のバイパス運転時の比消費電力,W/(m<sup>3</sup>/h)    

In [6]:
def get_SFP_bypass_d_t(f_SFP, f_SFP_bypass, HeatingSeason_d):
    """バイパス運転機能有する換気設備の時刻別比消費電量を計算する
    
    Parameters
    ----------
    f_SFP : float
        比消費電力,W/(m^3/h)  
    
    f_SFP_bypass : float
        バイパス運転時の比消費電力,W/(m^3/h)  
        
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
    
    Returns
    ----------
    SFP_bypass_d_t : ndarray(N-dimensional array)
        1年間の全時間の比消費電力量を格納したND配列, kWh
        d日t時の消費電力量が年開始時から8760個連続して格納されている
    """
    
    SFP_bypass_d_t = np.where(np.repeat(HeatingSeason_d, 24) == True, f_SFP, f_SFP_bypass)
    
    return SFP_bypass_d_t

In [7]:
get_SFP_bypass_d_t(0.3, 0.5, np.tile([True, False, True, False, True], 73))

array([ 0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,
        0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.5,  0.5,  0.5,  0.5,  0.5,  0.5,  0

## 温度差利用型ハイブリッド運転を有する換気設備の比消費電力

### 時刻別比消費電力
$$
f_{ SFP, hybrid-temperature, d, t }
=
\begin{cases}
0
& ,
EnableHybrid_{ d, t } = \mbox{True}
\\
f_{ SFP }
& ,
EnableHybrid_{ d } = \mbox{False}
\end{cases}
$$

$ EnableHybrid_{ d, t } $：日付dの時刻tにおける温度差利用型ハイブリッド運転が有効であるか否かの判定,True/False

In [38]:
def get_SFP_hybrid_temperature_d_t(f_SFP, \
                                   HeatingSeason_d, \
                                   area_number, \
                                   total_floor_area, \
                                   val_max, tolerance, maximum_iterations, \
                                   stories, \
                                   room_air_temperature, \
                                   c_value, \
                                   ventilation_type):
    
    """温度差利用ハイブリッド運転機能有する換気設備の時刻別比消費電量を計算する
    
    Parameters
    ----------
    f_SFP : float
        比消費電力,W/(m^3/h)  
    
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
    
    area_number : int
        地域区分番号,--  
    
    total_floor_area : float
        延べ床面積,m^2
        
    val_min : float
        二分法における初項（最小値）,Pa  
    
    val_max : float
        二分法における初項（最大値）,Pa  
    
    tolerance : float
        二分法における許容誤差,Pa  

    maximum_iterations : float
        二分法における繰り返し計算の上限回数,回  
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
    
    room_air_temperature : float
        室温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
    
        
    Returns
    ----------
    SFP_hybrid_temperature_d_t : ndarray(N-dimensional array)
        1年間の全時間の比消費電力量を格納したND配列, kWh
        d日t時の消費電力量が年開始時から8760個連続して格納されている
    """
        
    outside_air_temperature_d_t = import_outside_air_temperature_d_t(area_number)
    
    enable_hybrid_d_t = get_enable_hybrid_d_t(
                            total_floor_area, \
                            outside_air_temperature_d_t, \
                            HeatingSeason_d, \
                            val_max, tolerance, maximum_iterations, \
                            stories, \
                            room_air_temperature, \
                            c_value, \
                            total_floor_area, \
                            ventilation_type)
    
    SFP_hybrid_temperature_d_t = np.where(enable_hybrid_d_t == True, 0, f_SFP)
    
    return SFP_hybrid_temperature_d_t

In [20]:
get_SFP_hybrid_temperature_d_t(0.3)

array([ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,
        0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0.3,  0.3,  0.3,  0.3,  0.3,  0.3,  0

In [43]:
def get_enable_hybrid_d_t(total_floor_area, \
                          outside_air_temperature_d_t, \
                          HeatingSeason_d, \
                          val_max, tolerance, maximum_iterations, \
                          stories, \
                          room_air_temperature, \
                          c_value, \
                          ventilation_type):
    """温度差利用ハイブリッド運転が有効になる時刻を判定する。
    
    Parameters
    ----------
    total_floor_area : float
        延べ床面積,m^2  
    
    outside_air_temerature_d_t : ndarray(N-dimensional array)
        1年間の全時間の外気温度を格納したND配列, ℃
        d日t時の可否判定が年開始時から8760個連続して格納されている
        
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
        
    val_min : float
        二分法における初項（最小値）,Pa  
    
    val_max : float
        二分法における初項（最大値）,Pa  
    
    tolerance : float
        二分法における許容誤差,Pa  

    maximum_iterations : float
        二分法における繰り返し計算の上限回数,回  
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
    
    room_air_temperature : float
        室温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
        
    Returns
    ----------
    enable_hybrid_d_t : ndarray(N-dimensional array)
        1年間の全時間の温度差利用型ハイブリッド運転の可否判定を格納したND配列, True/False
        d日t時の可否判定が年開始時から8760個連続して格納されている
    """
    ventilation_volume = total_floor_area * 2.4 * 0.5
    
    natural_ventilation_volume_d_t = get_natural_ventilation_volume_d_t(
                                         outside_air_temperature_d_t, \
                                         HeatingSeason_d \
                                         -10**4, 10**4, 10**0, 10**4, \
                                         stories, \
                                         room_air_temperature, \
                                         c_value, \
                                         total_floor_area, \
                                         ventilation_type \
                                     )
    
    enable_hybrid_d_t =  \
        np.where(natural_ventilation_volume_d_t >= ventilation_volume, True, False)
    
    return enable_hybrid_d_t

In [44]:
def get_natural_ventilation_volume_d_t(outside_air_temperature_d_t, \
                                       HeatingSeason_dval_min, \
                                       val_max, tolerance, maximum_iterations, \
                                       stories, \
                                       room_air_temperature, \
                                       c_value, \
                                       total_floor_area, \
                                       ventilation_type):
    """温度差利用時の時刻別自然換気量を計算する。
    
    Parameters
    ----------
    outside_air_temerature_d_t : ndarray(N-dimensional array)
        1年間の全時間の外気温度を格納したND配列, ℃
        d日t時の可否判定が年開始時から8760個連続して格納されている
        
    HeatingSeason_d : ndarray(N-dimensional array)
        1年間の日別の暖房期間の判断を格納したND配列, h
        d日が暖房期間の場合True、それ以外の場合はFalseが格納されている。
    
    val_min : float
        二分法における初項（最小値）,Pa  
    
    val_max : float
        二分法における初項（最大値）,Pa  
    
    tolerance : float
        二分法における許容誤差,Pa  

    maximum_iterations : float
        二分法における繰り返し計算の上限回数,回  
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
    
    room_air_temperature : float
        室温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    total_floor_area : float
        延床面積,m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
    
    Returns
    ----------
    natural_ventilation_volume_d_t : ndarray(N-dimensional array)
        1年間の自然換気量を格納したND配列, m^3/h
        d日t時の自然換気量が年開始時から8760個連続して格納されている
    """
    
    heating_season_d_t = np.repeat(HeatingSeason_d, 24)
    
    natural_ventilation_volume_d_t = np.zeros(8760)
    
    for t in range(8760):
        
        if heating_season_d_t[t] == True:
            natural_ventilation_volume_d_t[t] = \
                get_natural_ventilation_volume(-10**4, 10**4, 10**0, 10**4, \
                                               stories, \
                                               room_air_temperature, \
                                               outside_air_temperature_d_t[t], \
                                               c_value, \
                                               total_floor_area, \
                                               ventilation_type)
        
    return natural_ventilation_volume_d_t

In [45]:
def get_natural_ventilation_volume(val_min, val_max, tolerance, maximum_iterations, \
                                    stories, \
                                    room_air_temperature, \
                                    outside_air_temperature, \
                                    c_value, \
                                    total_floor_area, \
                                    ventilation_type):
    """温度差利用時の自然換気量を収束計算（二分法）で求める。
    
    Parameters
    ----------
    val_min : float
        二分法における初項（最小値）,Pa  
    
    val_max : float
        二分法における初項（最大値）,Pa  
    
    tolerance : float
        二分法における許容誤差,Pa  

    maximum_iterations : float
        二分法における繰り返し計算の上限回数,回  
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
    
    room_air_temperature : float
        室温,℃  
    
    outside_air_temperature : float
        外気温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    total_floor_area : float
        延床面積,m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
    
    Returns
    ----------
    natural_ventilation_volume : float
        室内外温度差による自然換気量,m^3/h  
    """
    
    N_max = maximum_iterations
    TOL = tolerance
    
    x0 = val_min
    x1 = val_max
    
    n = 0
    
    while True:
        n += 1
        x = (x0 + x1)/2
        
        
        y0 = f(stories, x0, room_air_temperature, outside_air_temperature, c_value, total_floor_area, ventilation_type)
        y1 = f(stories, x1, room_air_temperature, outside_air_temperature, c_value, total_floor_area, ventilation_type)
        
        if y0 * y1 < 0:
            x1 = x
        else:
            x0 = x
        
        y = f(stories, x, room_air_temperature, outside_air_temperature, c_value, total_floor_area, ventilation_type)
        
        if f(x) == 0 or (x1 - x0)/2 < TOL or n >= N_max:
            break

    if n >= N_max:
        natural_ventilation_volume = 0
    else:
        natural_ventilation_volume = f(x)
        
    return natural_ventilation_volume

In [46]:
def f(stories, \
      pressure_difference, \
      room_air_temperature, \
      outside_air_temperature, \
      c_value, total_floor_area, \
      ventilation_type):
    """温度差利用時の自然換気量を求める(すきま風計算マクロ.xlsm 準拠)。
    
    Parameters
    ----------
        
    stories : int
        建物階数（1:平屋,2:2階建て,3:3階建て以上）,階  
        
    pressure_difference : float
        差圧,Pa  

    room_air_temperature : float
        室温,℃  
    
    outside_air_temperature : float
        外気温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    total_floor_area : float
        延床面積,m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
    
    Returns
    ----------
    Q_bal : float
        隙間風合計値,m^3/h  
    
    Q_in : float
        隙間風流入値,m^3/h  
    
    Q_out : float
        隙間風流出値,m^3/h  
    
    """
    
    if stories == 2:
        Q_bal, Q_in, Q_out = f_2stories(pressure_difference, \
                                        room_air_temperature, \
                                        outside_air_temperature, \
                                        c_value, \
                                        total_floor_area, \
                                        ventilation_type)
    
    return Q_bal, Q_in, Q_out

In [47]:
def f_2stories(pressure_difference, \
               room_air_temperature, \
               outside_air_temperature, \
               c_value, total_floor_area, \
               ventilation_type):
    """2階建ての温度差利用時の自然換気量を求める(すきま風計算マクロ.xlsm 準拠)。
    
    Parameters
    ----------
    pressure_difference : float
        差圧,Pa  
    
    room_air_temperature : float
        室温,℃  
    
    outside_air_temperature : float
        外気温,℃  
    
    c_value : float
        C値,cm^2/m^2  
    
    total_floor_area : float
        延床面積,m^2  
    
    ventilation_type : int
        換気種別（1:第一種,2:第二種,3:第三種）,-  
    
    Returns
    ----------
    Q_bal : float
        隙間風合計値,m^3/h  
    
    Q_in : float
        隙間風流入値,m^3/h  
    
    Q_out : float
        隙間風流出値,m^3/h  
    
    """

    
    
    t_i = room_air_temperature
    t_o = outside_air_temperature
    C_val = c_value
    A_floor = total_floor_area
    TypeVnt = ventilation_type
    
    ventilation_volume = A_floor * 2.4 * 0.5
    
    rho_i = 353.25 / (t_i + 273.15)
    rho_o = 353.25 / (t_o + 273.15)
    

    h0 = 0
    h1 = 1.6
    h2 = 2.4
    h3 = 2.9
    h4 = 4.5
    h5 = 5.3

    alphaA0 = C_val * A_floor / 4 / 10**4
    
    if TypeVnt == 1:
        alphaA1 = 0
    else:
        alphaA1 = ventilation_volume * 0.7 / 10**4 / 2
    
    alphaA2 = C_val * A_floor / 4 / 10**4
    alphaA3 = C_val * A_floor / 4 / 10**4
    
    if TypeVnt == 1:
        alphaA4 = 0
    else:
        alphaA4 = ventilation_volume * 0.7 / 10**4 / 2
    
    alphaA5 = C_val * A_floor / 4 / 10**4
    
    
    delta_p0 = pressure_difference
    delta_p1 = delta_p0 - 9.8 * h1 * ( rho_o - rho_i )
    delta_p2 = delta_p0 - 9.8 * h2 * ( rho_o - rho_i )
    delta_p3 = delta_p0 - 9.8 * h3 * ( rho_o - rho_i )
    delta_p4 = delta_p0 - 9.8 * h4 * ( rho_o - rho_i )
    delta_p5 = delta_p0 - 9.8 * h5 * ( rho_o - rho_i )
    
    if delta_p0 > 0:
        rho = rho_o
    else:
        rho = rho_i
    
    delta_p = abs(delta_p0)
    Q = alphaA0 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q0 = math.copysign(Q, delta_p0)
    
    delta_p = abs(delta_p1)
    Q = alphaA1 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q1 = math.copysign(Q, delta_p1)
    
    delta_p = abs(delta_p2)
    Q = alphaA2 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q2 = math.copysign(Q, delta_p2)
    
    delta_p = abs(delta_p3)
    Q = alphaA3 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q3 = math.copysign(Q, delta_p3)
    
    delta_p = abs(delta_p4)
    Q = alphaA4 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q4 = math.copysign(Q, delta_p4)
    
    delta_p = abs(delta_p5)
    Q = alphaA5 * math.sqrt( 2  * delta_p / rho ) * 3600
    Q5 = math.copysign(Q, delta_p5)
    
    Qs = np.array([Q0, Q1, Q2, Q3, Q4, Q5])
    
    print (Qs)
    
    
    Q_bal = np.sum(Qs)
    Q_in = np.sum(np.where(Qs < 0, 0, Qs))
    Q_out = np.sum(np.where(Qs > 0, 0, Qs))
    
    if TypeVnt == 2:
        Q_bal += ventilation_volume
    elif TypeVnt == 3:
        Q_bal -= ventilation_volume
    
    
    return Q_bal, Q_in, Q_out

In [48]:
f_2stories(5, 20, 0, 0.2, 120, 1)

[ 6.00638641  0.          4.59385109  4.24076044  0.          1.73517699]


(16.576174927144869, 16.576174927144869, 0.0)